In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import os
import glob
import sys

%matplotlib inline

# Image Data

In [5]:
image_train_data_queue = tf.train.string_input_producer(tf.train.match_filenames_once("./images/*.jpg"))
mask_train_data_queue = tf.train.string_input_producer(tf.train.match_filenames_once("./masks/*.jpg"))

image_reader = tf.WholeFileReader()

# Helper Methods